In [1]:
# Data must have the following characters.

# It must be a data frame;
# it must have only numeric data;
# it must have no missing values.
# Presumably the columns are PCA's.

# Also, Tune the hyperparameters in the last two paragraphs to your liking.

In [2]:
input_directory = 'C:/Users/jgbel/Desktop/data/Loan Defaulting/principal_component_matrix.pkl'
target_variable = "y"
target_variable_new_name = "y" # default value; you can leave it as 'y' if you'd like


In [3]:
import pandas as p
import xgboost as xgb
import numpy as n
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier


In [4]:
data_train = p.read_pickle(input_directory)
data_train.head(5)


,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,y
0,-6.637405,0.332877,3.170302,2.489962,-1.374604,0.516459,-1.725305,0.599770,0.829007,1.959165,...,-3.072154e-15,-4.998214e-16,-6.150163e-16,-1.677266e-15,8.763698e-16,-2.306176e-16,1.172465e-15,-9.561746e-16,-6.268549e-17,1
1,-1.189909,-0.175724,4.329798,-4.361114,0.238073,-0.268552,3.110056,-2.230101,-0.592611,-0.400393,...,7.228923e-14,1.697255e-14,2.978194e-14,2.843788e-14,-2.485141e-14,9.383686e-15,-5.725745e-14,1.889465e-14,-1.435737e-15,0
2,-0.140076,1.680802,-2.093459,0.193754,-1.992122,0.706007,-3.654360,1.011095,0.060673,-2.568054,...,-1.703271e-15,2.948890e-16,1.797595e-15,5.419774e-16,-5.116456e-15,-4.137084e-15,1.603608e-15,2.520989e-15,1.421932e-15,0
3,-0.160044,0.954807,-2.057069,0.235299,-1.183909,0.398971,0.546356,-0.932084,0.078123,0.425420,...,-2.516193e-15,-1.086418e-14,-2.892959e-15,8.944862e-15,6.724704e-15,5.490208e-15,-1.712270e-15,4.590015e-16,3.877390e-15,0
4,-0.111341,1.992211,-2.273490,-0.385983,0.085538,0.538493,-1.908771,-0.102221,-0.742868,0.138384,...,-3.159141e-15,-1.834589e-15,-7.205800e-15,1.007516e-14,-5.427343e-16,2.365505e-15,-2.706046e-15,-1.468450e-15,7.178776e-16,0


In [5]:
# Take a subset of the original data, to include only the top PCA's and the response variable.
data_train0 = data_train.iloc[:,0:20]
data_train0[target_variable_new_name] = data_train[target_variable]
data_train0.head(5)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,y
0,-6.637405,0.332877,3.170302,2.489962,-1.374604,0.516459,-1.725305,0.599770,0.829007,1.959165,...,2.086814,1.751076,-0.218232,-0.272763,2.929388,0.572855,-1.107051,0.458530,1.392028,1
1,-1.189909,-0.175724,4.329798,-4.361114,0.238073,-0.268552,3.110056,-2.230101,-0.592611,-0.400393,...,-0.855485,1.504215,1.579926,-0.343743,-1.520143,2.067731,0.073229,3.426459,-0.838725,0
2,-0.140076,1.680802,-2.093459,0.193754,-1.992122,0.706007,-3.654360,1.011095,0.060673,-2.568054,...,-0.754413,-0.940139,-0.176399,-0.499899,2.059499,1.986282,2.108539,-0.105842,-1.227604,0
3,-0.160044,0.954807,-2.057069,0.235299,-1.183909,0.398971,0.546356,-0.932084,0.078123,0.425420,...,0.106384,-1.609217,-0.984879,0.146607,1.199407,-0.114479,0.828936,-0.518632,-0.171251,0
4,-0.111341,1.992211,-2.273490,-0.385983,0.085538,0.538493,-1.908771,-0.102221,-0.742868,0.138384,...,-0.348457,-0.893077,-0.592483,0.434371,1.886537,2.114317,0.363315,0.627084,-0.373686,0


In [6]:
# Split the data into a train set and test set.
X_train, X_test, y_train, y_test = train_test_split(
    data_train0.drop(target_variable, axis = 1), 
    data_train0[target_variable],
    test_size = .15
)


In [7]:
# To create our model, we'll use a grid search to find the best hyperparameters.

# Create a dictionary of parameters to load into GridSearchCV().
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'booster': ['gbtree'],
              'objective': ['binary:logistic'],
              'learning_rate': [0.025], #so called `eta` value
              'max_depth': [2, 3, 4],
              'subsample': [0.1],
              'scale_pos_weight': [8],
              'colsample_bytree': [math.sqrt(data_train0.shape[1])/data_train0.shape[1]],
              'n_estimators': [5,10,15] #number of trees, change it to 1000 for better results,
              }
# Create a grid search object by specifying the model type, loading in the parameters sets above,
# and then passing in more parameters as constants if not already specified above.
grid = GridSearchCV(
    XGBClassifier(),
    parameters,
    n_jobs = 1,
    cv = 2
)
# Fit the grid to the training data and begin training.
grid.fit(X_train, y_train)
# Print out the best parameters.
grid.best_params_


{'booster': 'gbtree',
 'colsample_bytree': 0.21821789023599236,
 'learning_rate': 0.025,
 'max_depth': 2,
 'n_estimators': 15,
 'nthread': 4,
 'objective': 'binary:logistic',
 'scale_pos_weight': 8,
 'subsample': 0.1}

In [8]:
# Create a final model using optimal hyperparameter values.

# Specify the hyperparameter values for the final model.
model = XGBClassifier(
    nthread = 4,
    booster = 'gbtree',
    objective = 'binary:logistic',
    learning_rate = .025,
    max_depth = 6,
    subsample = .2,
    scale_pos_weight = 5.5,
    colsample_bytree = 0.21821789023599236,
    n_estimators = 1000,
    n_jobs = 1,
    cv = 2
)
# Train the model on the training data.
model.fit(X_train, y_train)
# Generate predictions on the test data.
predictions = model.predict(X_test)
df = p.DataFrame(
    {
        'y_Actual': y_test,
        'y_Predicted': predictions
    },
    columns=['y_Actual','y_Predicted']
)

# Print out confusion matrix report for the final model performance on the test data.
confusion_matrix = p.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)
print('\n\nSensitivity:  ',
      round(confusion_matrix[1][1]/(confusion_matrix[0][1] + confusion_matrix[1][1])*100, 2), '%', sep = "")
print('Specificity:  ',
      round(confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[1][0])*100, 2), '%', sep = "")

Predicted      0     1
Actual                
0          39265  3130
1           2956   776


Sensitivity:  20.79%
Specificity:  92.62%
